# Intro. to Snorkel: Extracting Spouse Relations from the News

## Part II: `Candidate` Extraction

In [1]:
%load_ext autoreload
%autoreload 2
import os

# TO USE A DATABASE OTHER THAN SQLITE, USE THIS LINE
# Note that this is necessary for parallel execution amongst other things...
# os.environ['SNORKELDB'] = 'postgres:///snorkel-intro'

from snorkel import SnorkelSession
session = SnorkelSession()

## Defining a `Candidate` schema
We now define the schema of the relation mention we want to extract (which is also the schema of the candidates).  This must be a subclass of `Candidate`, and we define it using a helper function. Here we'll define a binary _spouse relation mention_ which connects two `Span` objects of text.  Note that this function will create the table in the database backend if it does not exist:

In [2]:
from snorkel.models import candidate_subclass
Spouse = candidate_subclass('Spouse', ['person1', 'person2'])

## Writing a basic `CandidateExtractor`

Next, we'll write a basic function to extract **candidate spouse relation mentions** from the corpus.  The `SentenceParser` we used in Part I is built on [CoreNLP](http://stanfordnlp.github.io/CoreNLP/), which performs _named entity recognition_ for us.

We will extract `Candidate` objects of the `Spouse` type by identifying, for each `Sentence`, all pairs of ngrams (up to trigrams) that were tagged as people.  We do this with three objects:

* A `ContextSpace` defines the "space" of all candidates we even potentially consider; in this case we use the `Ngrams` subclass, and look for all n-grams up to 3 words long

* A `Matcher` heuristically filters the candidates we use.  In this case, we just use a pre-defined matcher which looks for all n-grams tagged by CoreNLP as "PERSON"

* A `CandidateExtractor` combines this all together!

In [3]:
from snorkel.candidates import Ngrams, CandidateExtractor
from snorkel.matchers import PersonMatcher

ngrams         = Ngrams(n_max=3)
person_matcher = PersonMatcher(longest_match_only=True)
cand_extractor = CandidateExtractor(Spouse, 
                                    [ngrams, ngrams], [person_matcher, person_matcher],
                                    symmetric_relations=False)

## Loading `Sentences` and splitting by `Document`

We will also _filter out_ sentences that mention at least five people:

In [4]:
def number_of_people(sentence):
    active_sequence = False
    count = 0
    for tag in sentence.ner_tags:
        if tag == 'PERSON' and not active_sequence:
            active_sequence = True
            count += 1
        elif tag != 'PERSON' and active_sequence:
            active_sequence = False
    return count

We'll split the documents 90 / 5 / 5 into train / dev / test splits as is standard.  Note that here, we'll do this in non-random order to preserve the splits that we already labeled, and we'll reference them by 0 / 1 / 2 respectively.


In [5]:
from snorkel.models import Document

docs = session.query(Document).order_by(Document.name).all()
ld   = len(docs)

train_sents = set()
dev_sents   = set()
test_sents  = set()
splits = (0.8, 0.9) if 'CI' in os.environ else (0.9, 0.95)
for i,doc in enumerate(docs):
    for s in doc.sentences:
        if number_of_people(s) < 5:
            if i < splits[0] * ld:
                train_sents.add(s)
            elif i < splits[1] * ld:
                dev_sents.add(s)
            else:
                test_sents.add(s)

## Running the `CandidateExtractor`

We run the `CandidateExtractor` by calling extract with the contexts to extract from, a name for the `CandidateSet` that will contain the results, and the current session.

In [6]:
%time cand_extractor.apply(train_sents, split=0)

Clearing existing...
Running UDF...
[========================================] 100%%

CPU times: user 1min 12s, sys: 1.52 s, total: 1min 14s
Wall time: 1min 13s


Here we specified that these `Candidates` belong to the training set by specifying `split=0`; recall that we're referring to train / dev / test as splits 0 / 1 / 2.

Note also that again, we could have specified a `parallelism` parameter to execute in parralel, if we had a non-SQLite database set up. Now let's get the candidates we just extracted:

In [7]:
train_cands = session.query(Spouse).filter(Spouse.split == 0).all()
print "Number of candidates:", len(train_cands)

Number of candidates: 4780


## Using the `Viewer` to inspect candidates

Next, we'll use the `Viewer` class--here, specifically, the `SentenceNgramViewer`--to inspect the data.

It is important to note, our goal here is to **maximize the recall of true candidates** extracted, **not** to extract _only_ the correct candidates. Learning to distinguish true candidates from false candidates is covered in Tutorial 4.

First, we instantiate the `Viewer` object, which groups the input `Candidate` objects by `Sentence`:

In [ ]:
from snorkel.viewer import SentenceNgramViewer

# NOTE: This if-then statement is only to avoid opening the viewer during automated testing of this notebook
# You should ignore this!
import os
if 'CI' not in os.environ:
    sv = SentenceNgramViewer(train_cands[:300], session)
else:
    sv = None

Next, we render the `Viewer.

In [ ]:
sv

Note that we can **navigate using the provided buttons**, or **using the keyboard (hover over buttons to see controls)**, highlight candidates (even if they overlap), and also **apply binary labels** (more on where to use this later!).  In particular, note that **the Viewer is synced dynamically with the notebook**, so that we can for example get the `Candidate` that is currently selected. Try it out!

In [ ]:
if 'CI' not in os.environ:
    print unicode(sv.get_selected())

### Traversing the _Context Hierarchy_

As you have already probably observed, in Snorkel, all `Candidate`s are basically just tuples of `Context`-type objects--in this (and most) cases, `Span`s. Given a `Candidate`, we can easily access its `Context`s by the names you've given them, or as a list:

In [8]:
c = train_cands[0]
c.person1

Span("Johnson", sentence=20809, chars=[255,261], words=[44,44])

In [9]:
c.get_contexts()

(Span("Johnson", sentence=20809, chars=[255,261], words=[44,44]),
 Span("Corbyn", sentence=20809, chars=[0,5], words=[0,0]))

These `Context` objects are part of a hierarchy of `Context` objects in Snorkel.  In our case, this hierarchy consists of `Document`s, `Sentence`s, and `Span`s.  We can traverse this hierarchy by using the specific names--e.g., `doc.sentences`--or by using the generic `get_parent()` and `get_children()` methods:

In [10]:
span = c.get_contexts()[0]
print span
print span.get_parent()
print span.get_parent().get_parent()

Span("Johnson", sentence=20809, chars=[255,261], words=[44,44])
Sentence(Document 0671_02c3742b-73cd-4fd9-b774-b4c03b23820a, 17, u'Corbyn wrote: \u201cWhile there\u2019s nothing wrong with the forensic examination of any candidate\u2019s policies or record, there is something deeply disturbing about the way in which the media have obsessed themselves with Livingstone, while the shallow banality of Johnson\u2019s rather flimsy policy pronouncements remain unchallenged.\u201d')
Document 0671_02c3742b-73cd-4fd9-b774-b4c03b23820a


`Span`s and `Sentence`s have special attributes which you can explore further in the documentation, and in tutorial section 4, when you will use some of these to write labeling functions.  For example, we can get the raw text span, the words, and other token types comprising a span:

In [11]:
print span.get_span()
print span.get_attrib_tokens()
print span.get_attrib_tokens('pos_tags')

Johnson
[u'Johnson']
[u'NNP']


### Repeating for development and test corpora
Finally, we will rerun the same operations for the other two news corpora: development and test. All we do for each is load in the `Corpus` object, collect the `Sentence` objects, and run them through the `CandidateExtractor`.

In [12]:
%%time
for i, sents in enumerate([dev_sents, test_sents]):
    cand_extractor.apply(sents, split=i+1)
    print "Number of candidates:", session.query(Spouse).filter(Spouse.split == i+1).count()

Clearing existing...
Running UDF...
[========================================] 100%%

Number of candidates: 223
Clearing existing...
Running UDF...
[========================================] 100%%

Number of candidates: 279
CPU times: user 9.64 s, sys: 888 ms, total: 10.5 s
Wall time: 9.68 s


Next, in Part 3, we will annotate some candidates with labels so that we can evaluate performance.